<img src="http://www.exalumnos.usm.cl/wp-content/uploads/2015/06/Isotipo-Negro.gif" title="Title text" width="20%" height="20%" />


<hr style="height:2px;border:none"/>
<h1 align='center'> INF-477 Redes Neuronales Artificiales I-2018 </h1>

<H3 align='center'> Tarea 1 - Redes Neuronales y *Deep Learning* </H3>


<hr style="height:2px;border:none"/>

**a) Construya un dataframe con los datos a analizar y descríbalo brevemete. Además, realice la división de éste en los conjuntos de entrenamiento, validación y testeo correspondientes. Comente por qué se deben eliminar ciertas columnas**

Las columnas 'Unnamed: 0' y 'pubchem_id' se eliminan por no tener datos relevante para los conjuntos de entrenamiento, validación y testeo.


In [1]:
import pandas as pd
datos= pd.read_csv("EnergyMolecule/roboBohr.csv")
#print(datos.shape)
#datos.info()
#print(datos)
#print(datos.describe())
...
#print("Quitando filas \n")
datos.drop(columns=['Unnamed: 0','pubchem_id'],axis=1,inplace=True) # con la propiedad drop elimina columnas
#print(datos.shape)
#print(datos)
total=len(datos)
df_train=datos[:int(0.6*total)]                       #60% de los datos
df_val=datos[int(0.6*total):int(0.85*total)]          #25% de los datos
df_test=datos[int(0.85*total)::]                      #15% restante
print ("Imprimir dataset")
print (df_test)
#print (total)

Imprimir dataset
                0          1          2          3          4          5  \
13805   73.516695  20.698877  18.620662  17.810283  15.905550  15.843780   
13806   73.516695  20.679672  16.113738  13.684521  13.684316  13.650499   
13807   73.516695  20.757186  18.681807  14.912967  13.684817  13.683619   
13808  388.023441  46.598629  46.596068  41.499896  28.840634  19.258076   
13809   73.516695  18.649283  18.596148  16.906068  16.138929  13.732604   
13810   73.516695  20.686502  20.676114  18.387888  18.385507  18.380429   
13811   73.516695  23.631201  23.627404  18.594704  17.821936  15.732707   
13812  388.023441  46.636408  46.608269  41.480748  28.929084  19.874522   
13813   73.516695  20.580008  20.468204  18.667254  14.894277  13.756640   
13814  388.023441  46.652934  46.629332  46.550982  46.520705  41.493579   
13815  388.023441  46.655242  46.625060  46.547117  46.519552  41.491914   
13816   73.516695  20.747524  18.505571  18.490333  14.221084  13.65188

Explique por qué se aconseja dicho preprocesamiento

In [2]:
#Pre-processing#
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(df_train)
X_train_scaled = pd.DataFrame(scaler.transform(df_train),columns=df_train.columns)
X_val_scaled =  pd.DataFrame(scaler.transform(df_val),columns=df_val.columns)
X_test_scaled =  pd.DataFrame(scaler.transform(df_test),columns=df_test.columns)
...
y_train = df_train.pop('Eat').values.reshape(-1,1)
y_val = df_val.pop('Eat').values.reshape(-1,1)
...
X_train_scaled.drop(columns=['Eat'],axis=1,inplace=True)
X_val_scaled.drop(columns=['Eat'],axis=1,inplace=True)
X_test_scaled.drop(columns=['Eat'],axis=1,inplace=True)

In [3]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.optimizers import SGD
model = Sequential()
model.add(Dense(256, input_dim=X_train_scaled.shape[1], kernel_initializer='uniform',activation="sigmoid"))
model.add(Dense(1, kernel_initializer='uniform',activation="linear")) 
model.compile(optimizer=SGD(lr=0.01),loss='mean_squared_error')
history = model.fit(X_train_scaled, y_train, epochs=250, verbose=1, validation_data=(X_val_scaled, y_val))

Using TensorFlow backend.


Train on 9745 samples, validate on 4060 samples
Epoch 1/250
9745/9745 [==============================] - 5s 522us/step - loss: 1.4786 - val_loss: 0.5836
Epoch 2/250
9745/9745 [==============================] - 4s 458us/step - loss: 0.5992 - val_loss: 0.4668
Epoch 3/250
9745/9745 [==============================] - 4s 359us/step - loss: 0.4922 - val_loss: 0.3604
Epoch 4/250
9745/9745 [==============================] - 4s 387us/step - loss: 0.4115 - val_loss: 0.3223
Epoch 5/250
9745/9745 [==============================] - 4s 400us/step - loss: 0.3560 - val_loss: 0.2935
Epoch 6/250
9745/9745 [==============================] - 4s 402us/step - loss: 0.3123 - val_loss: 0.3161
Epoch 7/250
9745/9745 [==============================] - 3s 355us/step - loss: 0.2710 - val_loss: 0.2362
Epoch 8/250
9745/9745 [==============================] - 4s 383us/step - loss: 0.2390 - val_loss: 0.2275
Epoch 9/250
9745/9745 [==============================] - 4s 451us/step - loss: 0.2154 - val_loss: 0.4119
Epoch 1

Epoch 78/250
9745/9745 [==============================] - 4s 415us/step - loss: 0.0297 - val_loss: 0.0462
Epoch 79/250
9745/9745 [==============================] - 5s 480us/step - loss: 0.0280 - val_loss: 0.0454
Epoch 80/250
9745/9745 [==============================] - 4s 435us/step - loss: 0.0296 - val_loss: 0.0426
Epoch 81/250
9745/9745 [==============================] - 4s 447us/step - loss: 0.0281 - val_loss: 0.0448
Epoch 82/250
9745/9745 [==============================] - 4s 447us/step - loss: 0.0284 - val_loss: 0.0585
Epoch 83/250
9745/9745 [==============================] - 5s 481us/step - loss: 0.0279 - val_loss: 0.0445
Epoch 84/250
9745/9745 [==============================] - 4s 435us/step - loss: 0.0280 - val_loss: 0.0442
Epoch 85/250
9745/9745 [==============================] - 4s 438us/step - loss: 0.0275 - val_loss: 0.0521
Epoch 86/250
9745/9745 [==============================] - 4s 391us/step - loss: 0.0277 - val_loss: 0.0429
Epoch 87/250
9745/9745 [======================

Epoch 155/250
9745/9745 [==============================] - 6s 602us/step - loss: 0.0193 - val_loss: 0.0408
Epoch 156/250
9745/9745 [==============================] - 6s 607us/step - loss: 0.0193 - val_loss: 0.0476
Epoch 157/250
9745/9745 [==============================] - 5s 482us/step - loss: 0.0190 - val_loss: 0.0414
Epoch 158/250
9745/9745 [==============================] - 5s 557us/step - loss: 0.0195 - val_loss: 0.0415
Epoch 159/250
9745/9745 [==============================] - 4s 415us/step - loss: 0.0185 - val_loss: 0.0393
Epoch 160/250
9745/9745 [==============================] - 3s 350us/step - loss: 0.0186 - val_loss: 0.0377
Epoch 161/250
9745/9745 [==============================] - 3s 356us/step - loss: 0.0186 - val_loss: 0.0369
Epoch 162/250
9745/9745 [==============================] - 4s 379us/step - loss: 0.0188 - val_loss: 0.0380
Epoch 163/250
9745/9745 [==============================] - 4s 386us/step - loss: 0.0185 - val_loss: 0.0364
Epoch 164/250
9745/9745 [============